In [1]:
# hide
# default_exp utils.cli
from nbdev.showdoc import *
from dsblocks.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# cli tools

> CLI tools

In [15]:
#export
from nbdev.imports import *
from nbdev.export import *
from nbdev.sync import *
from nbdev.merge import *
from nbdev.export2html import *
from nbdev.clean import *
from nbdev.test import *
from fastcore.script import *

In [ ]:
#for tests
import os

In [16]:
# export
@call_parse
def dsblocks_install_git_hooks():
    "Install git hooks to clean/trust notebooks automatically"
    try: path = get_config().config_file.parent
    except: path = Path.cwd()
    hook_path = path/'.git'/'hooks'
    fn = hook_path/'pre-push'
    hook_path.mkdir(parents=True, exist_ok=True)
    
    fn.write_text("""#!/usr/bin/bash

echo "\n==================== pre-push hook ===================="

# Export conda environment to yaml file
conda env export > env.yml

# Check if new environment file is different from original 
git diff --exit-code --quiet env.yml 

# If new environment file is different, commit it
if [[ $? -eq 0 ]]; then
    echo "Conda environment not changed. No additional commit."
else
    echo "Conda environment changed. Commiting new env.yml"
    git add env.yml
    git commit -m "Updating conda environment"
    echo 'You need to push again to push additional "Updating conda environment" commit.'
    exit 1
fi
""")
    
    os.chmod(fn, os.stat(fn).st_mode | stat.S_IEXEC)
    
    fn = hook_path/'post-merge'
    hook_path.mkdir(parents=True, exist_ok=True)
    
    fn.write_text("""#!/usr/bin/bash

echo "\n==================== post-merge hook ===================="

changed_files="$(git diff-tree -r --name-only --no-commit-id ORIG_HEAD HEAD)"

check_run() {
    echo "$changed_files" | grep --quiet "$1" && eval "$2"
}

echo "Have to update the conda environment"
check_run env.yml "conda env update --file env.yml"
""")
    
    os.chmod(fn, os.stat(fn).st_mode | stat.S_IEXEC)

### Usage example

In [17]:
# export tests.utils.test_cli  
def test_dsblocks_install_git_hooks ():
    if os.path.exists('.git/hooks/pre-push'):
        os.remove ('.git/hooks/pre-push')

    if os.path.exists('.git/hooks/post-merge'):
        os.remove ('.git/hooks/post-merge')
    
    dsblocks_install_git_hooks ()
    assert {'post-merge', 'pre-push'}.issubset(os.listdir ('.git/hooks'))
    with open('.git/hooks/pre-push', 'rt') as f: txt = f.read ()
    assert txt.startswith ('#!/usr/bin/bash\n\necho')
    assert txt.endswith ('fi\n')

    with open('.git/hooks/post-merge', 'rt') as f: txt = f.read ()
    assert txt.startswith ('#!/usr/bin/bash\n\necho')
    assert txt.endswith ('le env.yml"\n')
    
    os.remove ('.git/hooks/pre-push')
    os.remove ('.git/hooks/post-merge')

In [18]:
tst.run (test_dsblocks_install_git_hooks, tag='dummy')

running test_dsblocks_install_git_hooks
